In [2]:
import torch
import tiktoken

In [3]:
with open ("Amontillado.txt", "r") as f:
    raw_text = f.read()

raw_text[:50]

'The thousand injuries of Fortunato I had borne as '

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")


In [5]:
enc_text = tokenizer.encode(raw_text)

In [6]:
print(enc_text[:20])

[464, 7319, 6821, 286, 376, 1922, 5549, 314, 550, 28068, 355, 314, 1266, 714, 26, 475, 618, 339, 44716, 2402]


In [7]:
print(tokenizer.decode(enc_text[:20]))

The thousand injuries of Fortunato I had borne as I best could; but when he ventured upon


In [8]:
for i in range(1,10):
    print("INPUT:", tokenizer.decode(enc_text[:i]), "TARGET:", tokenizer.decode([enc_text[i]]))

INPUT: The TARGET:  thousand
INPUT: The thousand TARGET:  injuries
INPUT: The thousand injuries TARGET:  of
INPUT: The thousand injuries of TARGET:  F
INPUT: The thousand injuries of F TARGET: ortun
INPUT: The thousand injuries of Fortun TARGET: ato
INPUT: The thousand injuries of Fortunato TARGET:  I
INPUT: The thousand injuries of Fortunato I TARGET:  had
INPUT: The thousand injuries of Fortunato I had TARGET:  borne


## Data Set 

In [9]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]



## Dataloader

In [10]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [15]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


Inputs:
 tensor([[  464,  7319,  6821,   286],
        [  376,  1922,  5549,   314],
        [  550, 28068,   355,   314],
        [ 1266,   714,    26,   475],
        [  618,   339, 44716,  2402],
        [13277,    11,   314, 19982],
        [15827,    13,   921,    11],
        [  508,   523,   880,   760]])

Targets:
 tensor([[ 7319,  6821,   286,   376],
        [ 1922,  5549,   314,   550],
        [28068,   355,   314,  1266],
        [  714,    26,   475,   618],
        [  339, 44716,  2402, 13277],
        [   11,   314, 19982, 15827],
        [   13,   921,    11,   508],
        [  523,   880,   760,   262]])


In [18]:
type(inputs)

torch.Tensor

In [ ]:
#Gives dimensions of the tensor
inputs.shape

torch.Size([8, 4])

In [ ]:
#Look at the 2nd row
inputs[1]

tensor([ 376, 1922, 5549,  314])

In [ ]:
# Look at the first column, first element of each row
inputs[:,0]

tensor([  464,   376,   550,  1266,   618, 13277, 15827,   508])

In [ ]:
# Look at the ID in the 2nd row and 3rd column
inputs[1,2]

tensor(5549)

In [ ]:
#Print tensor row by row, converting to list so tokenizer can decode
for row in inputs:
    print(tokenizer.decode(row.tolist()))

The thousand injuries of
 Fortunato I
 had borne as I
 best could; but
 when he ventured upon
 insult, I vowed
 revenge. You,
 who so well know


## Example of Generating Embedding Vectors

In [ ]:
#illustrative example of how to generate the vectors of length 3 for the 6 word vocabular
vocab_size = 6
output_dim = 3

In [ ]:
#nn is a sub-module for nueral networks in torch which contains Embedding
#Embedding created embedding vectors of a certain length for a certain vocab size
embedding = torch.nn.Embedding(vocab_size, output_dim)

In [32]:
print(embedding.weight)

Parameter containing:
tensor([[-0.8602, -0.3075, -0.8975],
        [-0.5788, -0.9593,  0.5439],
        [ 0.2293,  0.3508, -0.3485],
        [ 1.3905,  0.6468,  0.6557],
        [ 1.1201, -0.1762,  0.6243],
        [ 0.4526,  0.7582,  0.6036]], requires_grad=True)


In [33]:
embedding.weight[0]

tensor([-0.8602, -0.3075, -0.8975], grad_fn=<SelectBackward0>)

In [ ]:
#get rid of reqires_grad bit of text
embedding.weight.data

tensor([[-0.8602, -0.3075, -0.8975],
        [-0.5788, -0.9593,  0.5439],
        [ 0.2293,  0.3508, -0.3485],
        [ 1.3905,  0.6468,  0.6557],
        [ 1.1201, -0.1762,  0.6243],
        [ 0.4526,  0.7582,  0.6036]])

In [ ]:
# Or get rid of it using detach()
print(embedding.weight.detach())

tensor([[-0.8602, -0.3075, -0.8975],
        [-0.5788, -0.9593,  0.5439],
        [ 0.2293,  0.3508, -0.3485],
        [ 1.3905,  0.6468,  0.6557],
        [ 1.1201, -0.1762,  0.6243],
        [ 0.4526,  0.7582,  0.6036]])


In [36]:
#What it would really look like with the correct sizes
#embedding = torch.nn.Embedding(tokenizer.n_vocab, 256)

In [40]:
A = embedding.weight.data
A

tensor([[-0.8602, -0.3075, -0.8975],
        [-0.5788, -0.9593,  0.5439],
        [ 0.2293,  0.3508, -0.3485],
        [ 1.3905,  0.6468,  0.6557],
        [ 1.1201, -0.1762,  0.6243],
        [ 0.4526,  0.7582,  0.6036]])

In [44]:
#making tensors from scratch-- just put in the elements
x = torch.tensor([2.1,1.3])
y = torch.tensor([1.5,2.7])
print(x,y)

tensor([2.1000, 1.3000]) tensor([1.5000, 2.7000])


In [ ]:
#computing the dot product if two tensors
torch.dot(x,y)

tensor(6.6600)